In [18]:
import numpy as np
import pandas as pd
import os
import path
import random
import glob
import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. Data loading from CSV files

In [19]:
df_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")


In [20]:
params = {
    'folder_dir': '../input/petfinder-pawpularity-score/',
    'image_dir': '../input/petfinder-pawpularity-score/train/',
    'img_size' : 384,
    'test_img_dir': '../input/petfinder-pawpularity-score/test',
    'device' : 'gpu'
}

**Augmentation**

In [21]:
def Transform_data(DIM = params['img_size']):
    return albumentations.Compose(
        [
            albumentations.Resize(DIM, DIM),
            albumentations.Normalize(
                mean = [0.485, 0.456, 0.406],
                std = [0.229, 0.224, 0.225]
            ),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit = 180, p=0.7),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2,
                val_shift_limit=0.2, p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit = (-0.1, 0.1),
                contrast_limit = (-0.1, 0.1), p=0.5
            ),
            ToTensorV2(p=1.0)
        ]
    )

In [22]:
def Transform_val(DIM = params['img_size']):
    return albumentations.Compose(
        [
            albumentations.Resize(DIM, DIM),
            albumentations.Normalize(
                mean = [0.485, 0.456, 0.406],
                std = [0.229, 0.224, 0.225]
            ),
            ToTensorV2(p=1.0)
        ]
    )

**Data preprocessing**

In [23]:
class PawDataSet():
    def __init__(self,dataset, params, transform = None):
        self.dataset = dataset
        self.image_path = dataset['Id'].apply(lambda x: os.path.join(params['image_dir'],f'{x}.jpg'))
        self.target_label = dataset['Pawpularity']
        self.transform = transform
        self.params = params

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_filepath = self.image_path[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        label = torch.tensor(self.target_label[idx]).float()
        return image, label


In [24]:
class TestDataSet():
    def __init__(self,dataset, params, transform = None):
        self.dataset = dataset
        self.image_path = dataset['Id'].apply(lambda x: os.path.join(params['test_img_dir'],f'{x}.jpg'))
        self.transform = transform
        self.params = params

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_filepath = self.image_path[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        return image



In [25]:
train_df = PawDataSet(dataset = df_data, params=params, transform = Transform_data())
test_df = TestDataSet(dataset = df_test, params=params, transform = Transform_val())

In [26]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_df, batch_size=1, shuffle=True)

**Model**

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*46*46, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.drop = nn.Dropout(p=0.2)
        self.count = 1
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drop(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.squeeze(1)
        return x   
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features =1 
        for s in size:
            num_features *= s
        return num_features

In [28]:
net=Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=135424, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)

In [29]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001)

**Train**

In [30]:
running_loss = 0.0
for i, data in enumerate(train_loader):
    inputs, label = data
    inputs, label = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    preds = net(inputs)
    loss = criterion(preds, label)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    print('loss: %.3f' %(running_loss/64))
    running_loss=0.0

loss: 30.097
loss: 25.025
loss: 11.999
loss: 10.395
loss: 10.103
loss: 8.516
loss: 7.494
loss: 5.835
loss: 7.162
loss: 6.816
loss: 8.943
loss: 7.225
loss: 5.254
loss: 7.573
loss: 7.700
loss: 7.090
loss: 8.681
loss: 8.425
loss: 6.606
loss: 7.272
loss: 9.790
loss: 6.858
loss: 6.603
loss: 8.714
loss: 7.734
loss: 7.567
loss: 6.349
loss: 5.043
loss: 6.708
loss: 7.907
loss: 7.708
loss: 5.302
loss: 5.700
loss: 5.179
loss: 8.189
loss: 5.942
loss: 7.311
loss: 4.003
loss: 5.911
loss: 7.187
loss: 6.848
loss: 5.696
loss: 9.318
loss: 6.783
loss: 6.988
loss: 6.831
loss: 7.223
loss: 5.822
loss: 6.923
loss: 8.562
loss: 6.962
loss: 7.107
loss: 9.225
loss: 7.531
loss: 8.428
loss: 7.541
loss: 4.430
loss: 7.823
loss: 7.729
loss: 5.513
loss: 9.301
loss: 6.886
loss: 7.271
loss: 6.225
loss: 7.095
loss: 6.986
loss: 8.361
loss: 5.936
loss: 7.106
loss: 6.516
loss: 5.769
loss: 7.944
loss: 4.891
loss: 8.581
loss: 5.979
loss: 9.238
loss: 7.495
loss: 8.090


In [31]:
# for epoch in range(1):
    
#     running_loss = 0.0
#     for i, data in enumerate(train_loader):
#         inputs, label = data
#         inputs, label = data[0].to(device), data[1].to(device)
        
#         optimizer.zero_grad()
#         preds = net(inputs)
#         loss = criterion(preds, label)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
#         print('[%d, %5d] loss: %.3f' %(epoch+1, i+1, running_loss/64))
#         running_loss=0.0
        
# print('Finished training()'.format(device))

In [32]:
final_result = []
for i, data in enumerate(test_loader):
    image = data[0]
    image = image.unsqueeze(0)
    image = image.to(device)
    print(image.size())
    preds = net(image).detach().cpu()
    preds = preds.squeeze()
    preds = preds.tolist()
    final_result.append(preds)
    
print(final_result)

for i in final_result:
    print(i)

df_test["Pawpularity"] = final_result
df_test = df_test[['Id','Pawpularity']]
df_test.to_csv("submission.csv", index=False)

torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
torch.Size([1, 3, 384, 384])
[40.504920959472656, 40.86041259765625, 40.69053268432617, 40.88846969604492, 40.35063552856445, 40.47684097290039, 40.91842269897461, 40.537994384765625]
40.504920959472656
40.86041259765625
40.69053268432617
40.88846969604492
40.35063552856445
40.47684097290039
40.91842269897461
40.537994384765625


In [33]:
df_test

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.504921
1,43a2262d7738e3d420d453815151079e,40.860413
2,4e429cead1848a298432a0acad014c9d,40.690533
3,80bc3ccafcc51b66303c2c263aa38486,40.888470
4,8f49844c382931444e68dffbe20228f4,40.350636
5,b03f7041962238a7c9d6537e22f9b017,40.476841
6,c978013571258ed6d4637f6e8cc9d6a3,40.918423
7,e0de453c1bffc20c22b072b34b54e50f,40.537994
